# Data Prepatation
- this one uses the github repo of data, not the google sheet

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import re
import os
import pickle
import os.path
from datetime import datetime, date, time 
from time import strftime
import pyarrow
import json
from dateutil.parser import parse


In [2]:
import git

'''
Download github repo and extract csv's

https://github.com/CSSEGISandData/2019-nCoV
'''

REPO = 'https://github.com/CSSEGISandData/2019-nCoV.git'
TMP_FOLDER = '/tmp/corona/'
TMP_GIT = os.path.join(TMP_FOLDER, '2019-nCoV')
DATA = os.path.join(TMP_GIT, 'daily_case_updates')


def clone_repo(TMP_FOLDER, REPO):
    print('Cloning Data Repo...')
    git.Git(tmp_folder).clone(REPO)

# Create Tmp Folder
if not os.path.isdir(TMP_FOLDER):
    print('Creating folder...')
    print('...', TMP_FOLDER)
    os.mkdir(TMP_FOLDER)

#Check if repo exists
if not os.path.isdir(TMP_GIT):
    clone_repo(TMP_FOLDER, REPO)
else:
    #get up to date repo
    print('Deleting out of date repo...')
    os.system('rm -rf ' + str(TMP_GIT))
    clone_repo(TMP_FOLDER, REPO)

Cloning Data Repo...


NameError: name 'tmp_folder' is not defined

In [49]:

def clean_sheet_names(new_ranges):
    '''
    Get rid of the duplicate sheets, only take the sheets from the 
    latest point in the day
    '''
    indices = []
    
    # Remove all sheets that dont have a numeric header
    new_ranges = [x for x in new_ranges if re.search(r'\d', x)]
        
    #split the names to just get the date
    clean_new_ranges = new_ranges.copy()
    for i, x in enumerate(clean_new_ranges):
        clean_new_ranges[i] = x.split('_')[0]    
    
    #Get the index of the latest tab for each date
    for item in set(clean_new_ranges):
        indices.append(clean_new_ranges.index(item))

    clean_new_ranges = []
    # Return wanted tabs for the sheet extraction
    for index in sorted(indices):
        clean_new_ranges.append(new_ranges[index])
        
    return clean_new_ranges

In [50]:
# get file names from /daily_case_updates/
sheets = os.listdir(DATA)

for s in sorted(sheets):
    print(s)
print('\n')

.DS_Store
01-21-2020_2200.csv
01-22-2020_1200.csv
01-23-2020_1200.csv
01-24-2020_0000.csv
01-24-2020_1200.csv
01-25-2020_0000.csv
01-25-2020_1200.csv
01-25-2020_2200.csv
01-26-2020_1100.csv
01-26-2020_2300.csv
01-27-2020_0900.csv
01-27-2020_1900.csv
01-27-2020_2030.csv
01-28-2020_1300.csv
01-28-2020_1800.csv
01-28-2020_2300.csv
01-29-2020_1330.csv
01-29-2020_1430.csv
01-29-2020_2100.csv
01-30-2020_1100.csv
01-30-2020_2130.csv
01-31-2020_1400.csv
02-01-2020_1000.csv
02-01-2020_1800.csv
02-01-2020_2300.csv
02-02-2020_0500.csv
02-02-2020_1945.csv
02-02-2020_2100.csv
02-03-2020_1230.csv
02-03-2020_2140.csv
02-04-2020_0800.csv
02-04-2020_1150.csv
02-04-2020_2200.csv
02-05-2020_1220.csv
02-06-2020_1318.csv
02-06-2020_2005.csv
02-07-2020_2024.csv
02-08-2020_1024.csv
02-08-2020_2304.csv
02-09-2020_1030.csv
02-09-2020_2320.csv
02-10-2020_1030.csv
02-10-2020_1930.csv
README.md




In [108]:
'''
For assigning date by the time sheet name
'''
def fix_dates(tmp_df, file_name):
    file_name = file_name.split('_')[0]
    tmp_df['Last Update'] = file_name
    return tmp_df

In [109]:
# Clean the result to the sheet tabs we want
cleaned_sheets = clean_sheet_names(sorted(sheets, reverse=True))

keep_cols = ['Confirmed', 'Country/Region', 'Deaths', 'Last Update', 'Province/State', 'Recovered']
numeric_cols = ['Confirmed', 'Deaths', 'Recovered']
all_csv = []
# Import all CSV's
for file in sorted(cleaned_sheets):
    tmp_df = pd.read_csv(os.path.join(DATA, file), index_col=None, header=0)
    tmp_df = tmp_df[keep_cols]
    tmp_df[numeric_cols] = tmp_df[numeric_cols].fillna(0)
    tmp_df[numeric_cols] = tmp_df[numeric_cols].astype(int)
    
    tmp_df = fix_dates(tmp_df, file)
    
    all_csv.append(tmp_df)
    
df = pd.concat(all_csv, axis=0, ignore_index=True, sort=True)

In [134]:
for sheet in cleaned_sheets:
    print(sheet) 

02-10-2020_1930.csv
02-09-2020_2320.csv
02-08-2020_2304.csv
02-07-2020_2024.csv
02-06-2020_2005.csv
02-05-2020_1220.csv
02-04-2020_2200.csv
02-03-2020_2140.csv
02-02-2020_2100.csv
02-01-2020_2300.csv
01-31-2020_1400.csv
01-30-2020_2130.csv
01-29-2020_2100.csv
01-28-2020_2300.csv
01-27-2020_2030.csv
01-26-2020_2300.csv
01-25-2020_2200.csv
01-24-2020_1200.csv
01-23-2020_1200.csv
01-22-2020_1200.csv
01-21-2020_2200.csv


In [110]:
df.head(200)

,Confirmed,Country/Region,Deaths,Last Update,Province/State,Recovered
0,0,Mainland China,0,01-21-2020,Anhui,0
1,10,Mainland China,0,01-21-2020,Beijing,0
2,5,Mainland China,0,01-21-2020,Chongqing,0
3,17,Mainland China,0,01-21-2020,Guangdong,0
4,0,Mainland China,0,01-21-2020,Guangxi,0
...,...,...,...,...,...,...
195,3,Malaysia,0,01-25-2020,NaN,0
196,1423,Mainland China,76,01-26-2020,Hubei,44
197,146,Mainland China,0,01-26-2020,Guangdong,2
198,128,Mainland China,0,01-26-2020,Zhejiang,1


In [ ]:


def get_data(sheet_range):
    tmp_df = pd.DataFrame([])
    
    
    
    
    
#     # rows with no deaths and recovered vals have shorter lists
#     # impute missing values with zeros
#     for i, row in enumerate(values):
#         if len(row) < len(header):
#             extra_zeros = (len(header) - len(row))
#             values[i] += [0] * extra_zeros

#     # Create Dataframe
#     if not values:
#         print('No data found.')
#     else:
#         all_data = []
#         for col_id, col_name in enumerate(header):
#             column_data = []
#             for row in values:
#                 column_data.append(row[col_id])
#             ds = pd.Series(data=column_data, name=col_name)
#             all_data.append(ds)
#         tmp = pd.concat(all_data, axis=1)

#         tmp = fix_dates(tmp, sheet_range)
        
    print('...', sheet_range)
    return tmp

In [111]:
# Now that we have all the data we now need to clean it 
# - Fill null values
# - remore suspected values
# - change column names
def clean_data(tmp_df):
    if 'Demised' in tmp_df.columns:
        tmp_df.rename(columns={'Demised':'Deaths'}, inplace=True)

    if 'Country/Region' in tmp_df.columns:
        tmp_df.rename(columns={'Country/Region':'country'}, inplace=True)
    
    if 'Province/State' in tmp_df.columns:
        tmp_df.rename(columns={'Province/State':'province'}, inplace=True)
      
    if 'Last Update' in tmp_df.columns:
        tmp_df.rename(columns={'Last Update':'date'}, inplace=True)
        
    if 'Suspected' in tmp_df.columns:
        tmp_df = tmp_df.drop(columns='Suspected')

    for col in tmp_df.columns:
        tmp_df[col] = tmp_df[col].fillna(0)
    
    #Lower case all col names
    tmp_df.columns = map(str.lower, tmp_df.columns)    
    
    return tmp_df


In [112]:
cleaned_dataframes = []

print('Cleaning dataframes...')
df  = clean_data(df)

Cleaning dataframes...


In [113]:
df

,confirmed,country,deaths,date,province,recovered
0,0,Mainland China,0,01-21-2020,Anhui,0
1,10,Mainland China,0,01-21-2020,Beijing,0
2,5,Mainland China,0,01-21-2020,Chongqing,0
3,17,Mainland China,0,01-21-2020,Guangdong,0
4,0,Mainland China,0,01-21-2020,Guangxi,0
...,...,...,...,...,...,...
1221,1,US,0,02-10-2020,"Los Angeles, CA",0
1222,1,US,0,02-10-2020,"Madison, WI",0
1223,1,US,0,02-10-2020,"Orange, CA",0
1224,1,US,0,02-10-2020,"Seattle, WA",1


In [115]:
# sheets need to be sorted by date value
print('Sorting by date...')
df = df.sort_values('date')

Sorting by date...


In [117]:
df.head(40)

,confirmed,country,deaths,date,province,recovered
0,0,Mainland China,0,01-21-2020,Anhui,0
26,2,Thailand,0,01-21-2020,0,0
25,1,South Korea,0,01-21-2020,0,0
24,1,US,0,01-21-2020,Washington,0
23,1,Japan,0,01-21-2020,0,0
21,1,Mainland China,0,01-21-2020,Yunnan,0
20,2,Mainland China,0,01-21-2020,Tianjin,0
19,1,Taiwan,0,01-21-2020,Taiwan,0
18,2,Mainland China,0,01-21-2020,Sichuan,0
17,9,Mainland China,0,01-21-2020,Shanghai,0


In [119]:

'''
Get the difference of the sum totals for each
date and plot them on a trendline graph
'''
def get_new_cases(final_df, col):
    diff_list = []
    tmp_df_list = []
    df = final_df.copy()
    
    for column in ['confirmed', 'deaths', 'recovered']:
        df[column] = df[column].replace('', 0).astype(int)

    for i, day in enumerate(df.date.unique()):    
        tmp_df = df[df.date == day]
        tmp_df_list.append(tmp_df[col].sum())
        
        if i == 0:
            diff_list.append(tmp_df[col].sum())
        else:
            diff_list.append(tmp_df[col].sum() - tmp_df_list[i-1])
        
    return diff_list

print('Calculating dataframe for new cases...')
daily_cases_df = pd.DataFrame([])
daily_cases_df['new_confirmed_cases'] = get_new_cases(df, 'confirmed')
daily_cases_df['new_deaths'] = get_new_cases(df, 'deaths')
daily_cases_df['new_recoveries'] = get_new_cases(df, 'recovered')
daily_cases_df['date'] = df.date.unique()


Calculating dataframe for new cases...


In [121]:
for column in ['confirmed', 'deaths', 'recovered']:
    df[column] = df[column].replace('', 0).astype(int)

current_infected = pd.DataFrame([])
current_infected['currently_infected'] = df.groupby('date').confirmed.sum() - (df.groupby('date').deaths.sum() + df.groupby('date').recovered.sum())

In [122]:
current_infected['delta'] = (current_infected['currently_infected'] - df.groupby('date').confirmed.sum())

In [123]:
daily_cases_df = pd.merge(daily_cases_df, current_infected, how='outer', on='date')

In [124]:
daily_cases_df

,new_confirmed_cases,new_deaths,new_recoveries,date,currently_infected,delta
0,332,6,25,01-21-2020,301,-31
1,223,11,3,01-22-2020,510,-45
2,98,1,2,01-23-2020,605,-48
3,288,8,6,01-24-2020,879,-62
4,1078,30,13,01-25-2020,1914,-105
5,775,24,5,01-26-2020,2660,-134
6,1679,27,9,01-27-2020,4303,-170
7,1584,25,47,01-28-2020,5815,-242
8,1726,38,23,01-29-2020,7480,-303
9,1993,43,54,01-30-2020,9376,-400


In [125]:
#Create date of extraction folder
save_dir  = '../data/' + str(datetime.date(datetime.now()))

print('Saving to data subdirectory...')
print('...', save_dir)

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

Saving to data subdirectory...
... ../data/2020-02-11


In [128]:
print('Saving...')
file_name = 'agg_data_{}.parquet.gzip'.format(datetime.date(datetime.now()))
df.astype(str).to_parquet(os.path.join(save_dir, file_name), compression='gzip')
print('...', file_name)


csv_file_name = 'agg_data_{}.csv'.format(datetime.date(datetime.now()))
df.astype(str).to_csv(os.path.join(save_dir, csv_file_name))
print('...', csv_file_name)


new_case_file_name = 'trend_{}.csv'.format(datetime.date(datetime.now()))
daily_cases_df.astype(str).to_csv(os.path.join(save_dir, new_case_file_name))
print('...', new_case_file_name)

print('Done!')

Saving...
... agg_data_2020-02-11.parquet.gzip
... agg_data_2020-02-11.csv
... trend_2020-02-11.csv
Done!


In [31]:
new_cases_df

,new_confirmed_cases,new_deaths,new_recoveries,date
0,555,0,0,01/22/2020
1,98,0,0,01/23/2020
2,288,26,36,01/24/2020
3,1078,30,13,01/25/2020
4,775,24,5,01/26/2020
5,1679,27,9,01/27/2020
6,1584,25,47,01/28/2020
7,1726,38,23,01/29/2020
8,1993,43,54,01/30/2020
9,1598,46,65,01/31/2020
